# Time series classification/ anomaly detection

This notebook aims to develop a thorough understanding of the data and to extract and visualize key insights for subsequent analyses.

## Table of contents

1. Setup & Imports
2. Feature Engineering


## Setup & Imports

In [1]:
from utils.constants import spectral_bands
from utils.data_loader import DataLoader
from utils.calculate_indices import CalculateIndices
from utils.basic_analysis import BasicDataAnalysis

from utils.time_series_aggregate import TimeSeriesAggregate
from utils.correlation_analysis import CorrelationAnalysis
from utils.sits_outlier_cleaner import SITSOutlierCleaner
from utils.analysis_disturbed import DisturbedYearAnalysis
from utils.dimensionality_reduction import SITS_DimensionalityReduction


from utils.visualization_utils.visualization_histogram import HistogramDataVisualization
from utils.visualization_utils.visualization_spectral import SpectralBandPlotter
from utils.visualization_utils.visualization_time_series import (
    plot_date_diff_distribution,
)
from utils.visualization_utils.visualization_function import (
    plot_intervals_timestamps,
    plot_top_correlations,
    plot_autocorrelation_bands,
    plot_band_differences,
)
from utils.visualization_utils.visualization_anomaly_detection import (
    plot_with_outliers_subplot,
    plot_outlier_detection_grid,
)

## Load Dataset

In [2]:
dataloader = DataLoader()
df_base = dataloader.load_transform("../data/raw/raw_trainset.csv")

## Basic Dataset Analysis

In [3]:
basicanalysis = BasicDataAnalysis(df_base)
print(basicanalysis.get_dtypes())

time                datetime64[ns]
id                           int64
disturbance_year           float64
doy                        float64
b2                         float64
b3                         float64
b4                         float64
b5                         float64
b6                         float64
b7                         float64
b8                         float64
b8a                        float64
b11                        float64
b12                        float64
species                     object
dtype: object


In [4]:
basicanalysis.show_overview()

Number of rows: 3927272
Number of columns: 15
Unique IDs: 28212
Unique species: 7
Earliest time: 2017-01-01 00:00:00
Latest time: 2022-11-02 00:00:00
Is id + species unique? True


In [5]:
basicanalysis.get_description()

,time,id,disturbance_year,doy,b2,b3,b4,b5,b6,b7,b8,b8a,b11,b12
count,3927272,3.927272e+06,3.927272e+06,3.927272e+06,3.927272e+06,3.927272e+06,3.927272e+06,3.927272e+06,3.927272e+06,3.927272e+06,3.927272e+06,3.927272e+06,3.927272e+06,3.927272e+06
mean,2020-02-03 09:45:20.828504064,1.408771e+04,2.522813e+02,1.766685e+02,2.721160e+02,4.303420e+02,3.456789e+02,6.755292e+02,1.682321e+03,2.039359e+03,2.166000e+03,2.289724e+03,1.156046e+03,5.999256e+02
min,2017-01-01 00:00:00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,2018-09-21 00:00:00,7.040000e+03,0.000000e+00,1.060000e+02,1.560000e+02,2.810000e+02,1.750000e+02,4.600000e+02,1.268000e+03,1.528000e+03,1.629000e+03,1.740000e+03,7.280000e+02,3.350000e+02
50%,2020-03-11 00:00:00,1.406400e+04,0.000000e+00,1.760000e+02,2.230000e+02,3.700000e+02,2.570000e+02,5.880000e+02,1.547000e+03,1.856000e+03,1.971000e+03,2.091000e+03,1.008000e+03,4.800000e+02
75%,2021-06-15 00:00:00,2.113500e+04,0.000000e+00,2.450000e+02,3.210000e+02,5.020000e+02,3.980000e+02,7.820000e+02,1.923000e+03,2.326000e+03,2.462000e+03,2.608000e+03,1.438000e+03,7.070000e+02
max,2022-11-02 00:00:00,2.821200e+04,2.020000e+03,3.660000e+02,2.008700e+04,1.753400e+04,1.752400e+04,1.794000e+04,1.982700e+04,2.143900e+04,2.181100e+04,2.469400e+04,1.855400e+04,1.758700e+04
std,NaN,8.145143e+03,6.674802e+02,8.540810e+01,2.201566e+02,2.598807e+02,3.079291e+02,3.476790e+02,6.454289e+02,8.020606e+02,8.528701e+02,8.593425e+02,6.080103e+02,4.184314e+02


In [6]:
basicanalysis.get_missing_counts()

,column_name,missing_count
0,time,0
1,id,0
2,disturbance_year,0
3,doy,0
4,b2,0
5,b3,0
6,b4,0
7,b5,0
8,b6,0
9,b7,0


## Feature Engineering

### Basic Feature Engineering

In [7]:
df = dataloader.feature_extraction(df_base)

### Datetime Feature Engineering

In [8]:
df = dataloader.date_feature_extraction(df)

### Advanced Feature Engineering

In [ ]:
calcindices = CalculateIndices()
df = calcindices.add_all_indices(df)

## Exploratory Data Analysis

### Visualization of the base dataframe 

In [ ]:
hist = HistogramDataVisualization(df)
hist.plot_unique_ids("year")

<div class="alert-info">
The distribution over the years clearly shows that the frequencies fluctuate only within a very narrow range.
</div>

In [ ]:
hist.plot_unique_ids("month_num")

<div class="alert-info">
The distribution of the occurring IDs shows that in the winter months of January and December, there is a noticeably lower frequency compared to the other months. It is assumed that this is due to the preprocessing step, during which many cloudy images were removed. Therefore, for the subsequent analysis, the aggregation of the data should be performed using an appropriate resampling interval that ensures the results remain meaningful and representative.
</div>

In [ ]:
hist.plot_unique_ids("species")

<div class="alert-info">
The seven categories show a clear difference in their frequency of occurrence. In general, they represent five tree species (including a mixed category), one disturbed class, and one soil class.
</div>

In [ ]:
hist.plot_id_distribution()

In [ ]:
hist.plot_median_id_distribution()

In [ ]:
plot_date_diff_distribution(df)

<div class="alert-info">
The difference in the various time gaps has a median of 8 days but shows high variance. It becomes evident that time series contain only rarely occurring observations.
</div>

### Correlation Analysis

In [ ]:
correlation = CorrelationAnalysis(df)
corr_matrix = correlation.get_correlation_matrix()
corr_matrix

In [ ]:
correlation.plot_correlation_matrix()

In [ ]:
top_corr_df = correlation.get_top_correlations(top_n=10)
plot_top_correlations(top_corr_df)

<div class="alert-info">
In the Pearson correlation, we observe strong correlations forming between the indices; however, these are not very meaningful. Therefore, it is necessary to use other methods to capture the true relationships.
</div>


### Analysis of Distrubed

In [ ]:
analysis = DisturbedYearAnalysis(df)
analysis.summary_unique_years()

In [ ]:
analysis.plot_disturbed_counts()

In [ ]:
analysis.plot_disturbance_by_species()

<div class="alert-info">

- The distribution of disturbance year and the corresponding species can be observed, going back up to 30 years from the marking year.

- The affected species are **soil and disturbed**.
</div>

## Explore Spectral-Bands

In [ ]:
spectral = SpectralBandPlotter(df)
spectral.plot_all_years(sample_size=5000, showfliers=False)

In [ ]:
spectral.plot_species_season_distribution()

## Time Series Analysis

<div class="alert-info">

In the time series analysis, **ID10** is used as an example to demonstrate different aggregations and methods, which can later be applied to the entire dataset.
</div>

### Anomaly Detection

In [ ]:
id_df = df[df["id"] == 10]
cleaner = SITSOutlierCleaner()
cleaner.fit_transform(id_df, band_columns=spectral_bands)
df_with_any_flag = cleaner.add_any_outlier_flag()
plot_with_outliers_subplot(df_with_any_flag, spectral_bands)

In [ ]:
plot_outlier_detection_grid(df_with_any_flag, bands=spectral_bands)

In [ ]:
score = cleaner.remaining_outliers_ratio()
print(f"Remaining outlier ratio (IsolationForest): {score:.2%}")

ratio = cleaner.zscore_outlier_ratio()
print(f"Outlier ratio (Z-Score method): {ratio:.2%}")

<div class="alert-info">

**Interpretation**

Remaining outlier ratio (IsolationForest): Shows the percentage of records that the IsolationForest model still considers outliers after cleaning. A lower value means the cleaning was more effective in removing anomalies according to the model.

Outlier ratio (Z-Score method): Indicates the share of records that are statistical outliers (e.g., more than 3 standard deviations away from the mean in at least one feature). This gives a complementary, more “classical” perspective on how extreme your data points are.
</div>

### Time series progression

In [ ]:
cleaned_id_df = cleaner.get_interpolated_only()

In [ ]:
cleaned_id_df = cleaner.get_interpolated_only()
ts_agg = TimeSeriesAggregate(cleaned_id_df)
df_2w = ts_agg.aggregate_timeseries(freq="2W", method="mean")
dataloader = DataLoader()
df_2w_agg = dataloader.date_feature_extraction(df_2w)

In [ ]:
spectral = SpectralBandPlotter(df_2w_agg)
spectral.plot_spectral_development_over_years(addition="aggregated")

In [ ]:
plot_intervals_timestamps(df_2w_agg, addition="aggregated")

#### Autocorrelation

In [ ]:
plot_autocorrelation_bands(df_2w_agg, spectral_bands, lags=26)

<div class="alert-info">

**The ACF and PACF plots indicate that:**
- The time series exhibit short-term autocorrelation but no strong long-term or seasonal structure.
- The decay patterns and PACF cutoffs suggest that simple autoregressive models (e.g., AR(1)) may sufficiently describe the temporal dependencies in these variables.
- The data appear stationary, which is favorable for modeling approaches like ARIMA or LSTM without further differencing.
</div>

### Time Series Cycle Analysis

In [ ]:
plot_band_differences(df_2w_agg, shift_years=1)

<div class="alert-info">

**Seasonal Patterns:**

Both the original (blue) and shifted (red) time series show strong seasonal cycles.
Peaks and troughs repeat roughly on an annual basis, which is typical for vegetation or water-related indices in satellite data.

**Shifted Series Alignment:**

The shifted series (+1 year, red) generally aligns well with the previous year’s patterns.
There are slight offsets in peak timing in some years, indicating inter-annual variability.

**Differences (Green Dotted Line):**

The difference (Original − Shifted) fluctuates around zero but shows pronounced deviations during certain years.
Positive spikes indicate that the current year’s values are higher than the previous year at the same period.
Negative spikes indicate that the current year’s values are lower than the previous year.
</div>

### PCA-Analysis

In [ ]:
dr = SITS_DimensionalityReduction(method="PCA", n_components=3)
dr.fit(df, label_col="species")
dr.plot_2d()

<div class="alert-info">

- Clear separation of soil: **Soil points** are spread along PC1 (and partially PC2), indicating soil samples are clearly distinct from vegetation classes.
- Vegetation classes overlap: Tree species **(Norway spruce, Scots pine, oak, beech)** cluster near the center, showing similar feature patterns and no clear separation.
- Disturbed & mixed classes: These overlap strongly with other vegetation, suggesting no distinct patterns.
</div>

In [ ]:
df_sample = dataloader.get_sample(df, n_ids = 6000)
dr_tsne = SITS_DimensionalityReduction(method="tSNE", n_components=2, perplexity=50)
dr_tsne.fit(df_sample, label_col="species")
dr_tsne.plot_2d()

<div class="alert-info">

- Clear clusters: **soil** and **beech** form compact, well-separated clusters.
- Overlap: **Norway_spruce**, **Norway_spruce_mixed** and **Scots_pine** strongly overlap and are hard to separate.
- **Disturbed** partially mixes with conifer clusters.
- Summary: Some classes **(soil, beech)** are clearly separable, while conifer species show strong overlap.
</div>